In [2]:
%cd /content/drive/MyDrive/real_fake_face_detection

/content/drive/MyDrive/real_fake_face_detection


In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="5tzW6kQaNZO876hjqL3F")
project = rf.workspace("hithyshi-kandula-6fq72").project("forged-images")
version = project.version(3)
dataset = version.download("voc")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to forged-images-3 in voc:: 100%|██████████| 4083/4083 [00:46<00:00, 87.29it/s]


Split to train, test, valid

In [4]:
import os
import random
import shutil

# Directory paths
dataset_folder = '/content/drive/MyDrive/real_fake_face_detection/dataset/totalset'
train_folder = '/content/drive/MyDrive/real_fake_face_detection/dataset/train'
test_folder = '/content/drive/MyDrive/real_fake_face_detection/dataset/test'
valid_folder = '/content/drive/MyDrive/real_fake_face_detection/dataset/valid'

# Create folders if they don't exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)
os.makedirs(valid_folder, exist_ok=True)

# Function to list all image files and their types (real/fake)
def list_images_and_xml(dataset_folder):
    real_images = []
    fake_images = []

    for root, dirs, files in os.walk(dataset_folder):
        for file in files:
            if file.endswith('.jpg'):  # Adjust if your images have a different extension
                image_path = os.path.join(root, file)
                xml_file = os.path.splitext(file)[0] + '.xml'
                xml_path = os.path.join(root, xml_file)
                if os.path.exists(xml_path):
                    if 'real' in root.lower():  # Adjust if 'real' images are in a different folder
                        real_images.append((image_path, xml_path))
                    elif 'fake' in root.lower():  # Adjust if 'fake' images are in a different folder
                        fake_images.append((image_path, xml_path))

    return real_images, fake_images

# List all 'real' and 'fake' image and XML pairs
real_images, fake_images = list_images_and_xml(dataset_folder)

# Shuffle 'real' and 'fake' image pairs randomly
random.shuffle(real_images)
random.shuffle(fake_images)

# Calculate counts based on proportions
total_real_images = len(real_images)
total_fake_images = len(fake_images)

train_ratio = 0.78
test_ratio = 0.08
valid_ratio = 0.14

train_real_count = int(total_real_images * train_ratio)
test_real_count = int(total_real_images * test_ratio)
valid_real_count = total_real_images - train_real_count - test_real_count

train_fake_count = int(total_fake_images * train_ratio)
test_fake_count = int(total_fake_images * test_ratio)
valid_fake_count = total_fake_images - train_fake_count - test_fake_count

# Initialize counters
train_real_added = 0
test_real_added = 0
valid_real_added = 0

train_fake_added = 0
test_fake_added = 0
valid_fake_added = 0

# Distribute 'real' images and XML files into train, test, valid folders
for image_path, xml_path in real_images:
    if train_real_added < train_real_count:
        shutil.copy(image_path, os.path.join(train_folder, os.path.basename(image_path)))
        shutil.copy(xml_path, os.path.join(train_folder, os.path.basename(xml_path)))
        train_real_added += 1
    elif test_real_added < test_real_count:
        shutil.copy(image_path, os.path.join(test_folder, os.path.basename(image_path)))
        shutil.copy(xml_path, os.path.join(test_folder, os.path.basename(xml_path)))
        test_real_added += 1
    elif valid_real_added < valid_real_count:
        shutil.copy(image_path, os.path.join(valid_folder, os.path.basename(image_path)))
        shutil.copy(xml_path, os.path.join(valid_folder, os.path.basename(xml_path)))
        valid_real_added += 1

# Distribute 'fake' images and XML files into train, test, valid folders
for image_path, xml_path in fake_images:
    if train_fake_added < train_fake_count:
        shutil.copy(image_path, os.path.join(train_folder, os.path.basename(image_path)))
        shutil.copy(xml_path, os.path.join(train_folder, os.path.basename(xml_path)))
        train_fake_added += 1
    elif test_fake_added < test_fake_count:
        shutil.copy(image_path, os.path.join(test_folder, os.path.basename(image_path)))
        shutil.copy(xml_path, os.path.join(test_folder, os.path.basename(xml_path)))
        test_fake_added += 1
    elif valid_fake_added < valid_fake_count:
        shutil.copy(image_path, os.path.join(valid_folder, os.path.basename(image_path)))
        shutil.copy(xml_path, os.path.join(valid_folder, os.path.basename(xml_path)))
        valid_fake_added += 1

print(f"Dataset split completed: Train - {train_ratio*100}%, Test - {test_ratio*100}%, Valid - {valid_ratio*100}%")


Dataset split completed: Train - 78.0%, Test - 8.0%, Valid - 14.000000000000002%


Parsing xml files and extracting information, then finding required information such as center_x, center_y, w, h, label_id

In [5]:
import os
from xml.etree import ElementTree as ET
import pandas as pd

# Function to extract annotations from a folder and normalize paths
def extract_annotations_and_normalize(folder_path):
    data = []

    # Iterate over each XML file in the directory
    for xml_file in os.listdir(folder_path):
        if xml_file.endswith('.xml'):
            xml_path = os.path.join(folder_path, xml_file)
            xml_path = xml_path.replace('\\', '/')  # Convert backslashes to forward slashes
            tree = ET.parse(xml_path)
            root = tree.getroot()

            # Extract image metadata
            image_name = root.find('filename').text
            width = int(root.find('size').find('width').text)
            height = int(root.find('size').find('height').text)

            # Extract object annotations
            for obj in root.findall('object'):
                obj_name = obj.find('name').text
                bbox = obj.find('bndbox')
                xmin = int(bbox.find('xmin').text)
                ymin = int(bbox.find('ymin').text)
                xmax = int(bbox.find('xmax').text)
                ymax = int(bbox.find('ymax').text)

                # Calculate normalized coordinates
                center_x = ((xmax + xmin) / 2) / width
                center_y = ((ymax + ymin) / 2) / height
                w = (xmax - xmin) / width
                h = (ymax - ymin) / height

                # Perform label encoding
                if obj_name == 'real':
                    label_id = 0
                elif obj_name == 'fake':
                    label_id = 1
                else:
                    label_id = -1  # Handle unknown labels if needed

                data.append({
                    'filename': image_name,
                    'width': width,
                    'height': height,
                    'name': obj_name,
                    'xmin': xmin,
                    'xmax': xmax,
                    'ymin': ymin,
                    'ymax': ymax,
                    'center_x': center_x,
                    'center_y': center_y,
                    'w': w,
                    'h': h,
                    'label_id': label_id
                })

    return data

# Directory paths for train, test, and valid datasets
# train_folder = '/content/drive/MyDrive/real_fake_face_detection/dataset/train'
# test_folder = '/content/drive/MyDrive/real_fake_face_detection/dataset/test'
# valid_folder = '/content/drive/MyDrive/real_fake_face_detection/dataset/valid'

# Extract annotations and normalize paths for each dataset folder
train_data = extract_annotations_and_normalize(train_folder)
test_data = extract_annotations_and_normalize(test_folder)
valid_data = extract_annotations_and_normalize(valid_folder)

# Convert to DataFrames
train_df = pd.DataFrame(train_data, columns=['filename', 'width', 'height', 'name', 'xmin', 'xmax', 'ymin', 'ymax', 'center_x', 'center_y', 'w', 'h','label_id'])
test_df = pd.DataFrame(test_data, columns=['filename', 'width', 'height', 'name', 'xmin', 'xmax', 'ymin', 'ymax', 'center_x', 'center_y', 'w', 'h','label_id'])
valid_df = pd.DataFrame(valid_data, columns=['filename', 'width', 'height', 'name', 'xmin', 'xmax', 'ymin', 'ymax', 'center_x', 'center_y', 'w', 'h','label_id'])

# Display the first few rows of each DataFrame to verify
print("Train Dataset:")
print(train_df.head())

print("\nTest Dataset:")
print(test_df.head())

print("\nValidation Dataset:")
print(valid_df.head())


Train Dataset:
                                            filename  width  height  name  \
0  real_00087_jpg.rf.d799c059c565292290762a9f4c4d...    640     640  real   
1  real_00540_jpg.rf.ca09493e6aa3f456f65bdf528927...    640     640  real   
2  mid_250_1111_jpg.rf.fa719d502541af27a6103a846e...    640     640  fake   
3  hard_102_1001_jpg.rf.f0ebae1ff7a5f7bc0b6a1a4a7...    640     640  fake   
4  hard_195_1000_jpg.rf.e9895553d9d5d3a83db98248f...    640     640  fake   

   xmin  xmax  ymin  ymax  center_x  center_y         w         h  label_id  
0    98   549   182   637  0.505469  0.639844  0.704688  0.710938         0  
1   102   596    75   639  0.545312  0.557813  0.771875  0.881250         0  
2   102   553    27   625  0.511719  0.509375  0.704688  0.934375         1  
3    84   545    47   621  0.491406  0.521875  0.720313  0.896875         1  
4    71   520    38   620  0.461719  0.514062  0.701562  0.909375         1  

Test Dataset:
                                       

Save dataframes in .csv files for future use

In [6]:
# Save DataFrames to CSV files
train_df.to_csv('/content/drive/MyDrive/real_fake_face_detection/train.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/real_fake_face_detection/test.csv', index=False)
valid_df.to_csv('/content/drive/MyDrive/real_fake_face_detection/valid.csv', index=False)

In [7]:
# Define file paths for CSV files
train_csv = '/content/drive/MyDrive/real_fake_face_detection/train.csv'
test_csv = '/content/drive/MyDrive/real_fake_face_detection/test.csv'
valid_csv = '/content/drive/MyDrive/real_fake_face_detection/valid.csv'

# Load CSV files into DataFrames
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)
valid_df = pd.read_csv(valid_csv)

# Count 'real' and 'fake' instances in each DataFrame
train_counts = train_df['name'].value_counts()
test_counts = test_df['name'].value_counts()
valid_counts = valid_df['name'].value_counts()

# Display counts
print("Train Dataset:")
print(train_counts)

print("\nTest Dataset:")
print(test_counts)

print("\nValidation Dataset:")
print(valid_counts)

Train Dataset:
name
real    857
fake    745
Name: count, dtype: int64

Test Dataset:
name
fake    83
real    80
Name: count, dtype: int64

Validation Dataset:
name
real    151
fake    137
Name: count, dtype: int64


Conversion to text format to train using yolov5

In [8]:
# Function to convert CSV annotations to YOLO format
def csv_to_yolo(csv_file, image_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    df = pd.read_csv(csv_file)

    for index, row in df.iterrows():
        image_name = row['filename']
        image_path = os.path.join(image_folder, image_name)

        # Skip XML files
        if not image_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
            continue

        image_width = row['width']
        image_height = row['height']
        class_name = row['name']
        center_x = row['center_x']
        center_y = row['center_y']
        w = row['w']
        h = row['h']
        label_id = row['label_id']

        txt_filename = os.path.splitext(image_name)[0] + '.txt'
        txt_path = os.path.join(output_folder, txt_filename)

        with open(txt_path, 'a') as txt_file:
            # Write to YOLO format file
            txt_file.write(f"{label_id} {center_x} {center_y} {w} {h}\n")

# Directory paths
train_csv = '/content/drive/MyDrive/real_fake_face_detection/train.csv'
test_csv = '/content/drive/MyDrive/real_fake_face_detection/test.csv'
valid_csv = '/content/drive/MyDrive/real_fake_face_detection/valid.csv'

train_image_folder = '/content/drive/MyDrive/real_fake_face_detection/dataset/train'
test_image_folder = '/content/drive/MyDrive/real_fake_face_detection/dataset/test'
valid_image_folder = '/content/drive/MyDrive/real_fake_face_detection/dataset/valid'

train_output_folder = '/content/drive/MyDrive/real_fake_face_detection/dataset/train_txt'
test_output_folder = '/content/drive/MyDrive/real_fake_face_detection/dataset/test_txt'
valid_output_folder = '/content/drive/MyDrive/real_fake_face_detection/dataset/valid_txt'

# Create output folders if they don't exist
os.makedirs(train_output_folder, exist_ok=True)
os.makedirs(test_output_folder, exist_ok=True)
os.makedirs(valid_output_folder, exist_ok=True)

# Convert CSV to YOLO format for train dataset
csv_to_yolo(train_csv, train_image_folder, train_output_folder)

# Convert CSV to YOLO format for test dataset
csv_to_yolo(test_csv, test_image_folder, test_output_folder)

# Convert CSV to YOLO format for valid dataset
csv_to_yolo(valid_csv, valid_image_folder, valid_output_folder)

print("Conversion to YOLO format completed.")


Conversion to YOLO format completed.


transfering text files to image folders

In [9]:
import shutil

# Base directory
base_dir = '/content/drive/MyDrive/real_fake_face_detection/dataset'

# Function to move text files from text folders to corresponding image folders and ignore xml files
def move_text_files(split):
    image_folder = os.path.join(base_dir, split)
    text_folder = os.path.join(base_dir, f'{split}_txt')

    # Move .txt files from text_folder to images folder and ignore .xml files
    for file_name in os.listdir(text_folder):
        if file_name.endswith('.txt'):
            src_file = os.path.join(text_folder, file_name)
            dst_file = os.path.join(image_folder, file_name)

            try:
                # Move the file
                shutil.move(src_file, dst_file)
                print(f"Moved '{file_name}' to '{image_folder}'")
            except FileNotFoundError as e:
                print(f"Error moving '{file_name}': {e}")
            except Exception as e:
                print(f"Unexpected error moving '{file_name}': {e}")
        elif file_name.endswith('.xml'):
            # Ignore .xml files
            continue

# Move text files for train, valid, and test splits
move_text_files('train')
move_text_files('valid')
move_text_files('test')

print("All .txt files moved successfully.")


Moved 'real_00087_jpg.rf.d799c059c565292290762a9f4c4d05dd.txt' to '/content/drive/MyDrive/real_fake_face_detection/dataset/train'
Moved 'real_00540_jpg.rf.ca09493e6aa3f456f65bdf528927f02e.txt' to '/content/drive/MyDrive/real_fake_face_detection/dataset/train'
Moved 'mid_250_1111_jpg.rf.fa719d502541af27a6103a846ed71c4b.txt' to '/content/drive/MyDrive/real_fake_face_detection/dataset/train'
Moved 'hard_102_1001_jpg.rf.f0ebae1ff7a5f7bc0b6a1a4a7743c839.txt' to '/content/drive/MyDrive/real_fake_face_detection/dataset/train'
Moved 'hard_195_1000_jpg.rf.e9895553d9d5d3a83db98248f7ecc113.txt' to '/content/drive/MyDrive/real_fake_face_detection/dataset/train'
Moved 'real_00710_jpg.rf.c57dbdbbc76689a1b5a3932b06e3c2d3.txt' to '/content/drive/MyDrive/real_fake_face_detection/dataset/train'
Moved 'easy_56_1110_jpg.rf.4785985b87b5bebc3feee5c2f6b89325.txt' to '/content/drive/MyDrive/real_fake_face_detection/dataset/train'
Moved 'real_00118_jpg.rf.18eb481531db52405018e002fdf6aaee.txt' to '/content/driv

transferring all xml files to thier corresponding folders

In [10]:
import shutil

base_dir = '/content/drive/MyDrive/real_fake_face_detection/dataset'

def move_xml_files(split):
    image_folder = os.path.join(base_dir, split)
    xml_folder = os.path.join(base_dir, f'{split}_xml')

    count = 0
    for file_name in os.listdir(image_folder):
        if file_name.endswith('.xml'):  # Check if the file is an XML file
            src_file = os.path.join(image_folder, file_name)
            dst_file = os.path.join(xml_folder, file_name)

            try:
                if not os.path.exists(dst_file):  # Check if file already exists in destination
                    shutil.move(src_file, dst_file)
                    print(f"Moved '{file_name}' to '{xml_folder}'")
                    count += 1
                else:
                    print(f"Skipping '{file_name}' as it already exists in '{xml_folder}'")
            except FileNotFoundError as e:
                print(f"Error moving '{file_name}': {e}")
            except Exception as e:
                print(f"Unexpected error moving '{file_name}': {e}")

        elif file_name.endswith('.txt') or file_name.endswith('.jpg'):
            print(f"Ignoring '{file_name}' in '{image_folder}'")

    print(f"Moved {count} XML files successfully.")

# Move XML files for train, test, and valid splits
move_xml_files('train')
move_xml_files('test')
move_xml_files('valid')


Streaming output truncated to the last 5000 lines.
Moved 'real_00996_jpg.rf.4bd64811c1840fbc70cd1edd3a9b88ce.xml' to '/content/drive/MyDrive/real_fake_face_detection/dataset/train_xml'
Ignoring 'real_00296_jpg.rf.aea8b9597864bf4a36bf2f868d9dd0f2.jpg' in '/content/drive/MyDrive/real_fake_face_detection/dataset/train'
Moved 'real_00296_jpg.rf.aea8b9597864bf4a36bf2f868d9dd0f2.xml' to '/content/drive/MyDrive/real_fake_face_detection/dataset/train_xml'
Ignoring 'real_00848_jpg.rf.8e00e68d53b30b294a24bfdda98c5b94.jpg' in '/content/drive/MyDrive/real_fake_face_detection/dataset/train'
Moved 'real_00848_jpg.rf.8e00e68d53b30b294a24bfdda98c5b94.xml' to '/content/drive/MyDrive/real_fake_face_detection/dataset/train_xml'
Ignoring 'real_01010_jpg.rf.701c38bdc9c07de60b1b3ad9e648f0a2.jpg' in '/content/drive/MyDrive/real_fake_face_detection/dataset/train'
Moved 'real_01010_jpg.rf.701c38bdc9c07de60b1b3ad9e648f0a2.xml' to '/content/drive/MyDrive/real_fake_face_detection/dataset/train_xml'
Ignoring 'real